In [83]:
#data clean v2 Intentamos mejorar el accuracy con otro tipo de ingenieria de variables
import pandas as pd 
import numpy as np
import os 

In [84]:
os.chdir('/home/joseacentic/centic/automl/hackathon-inditex-data-recommender/data')
products = pd.read_pickle('./raw/products.pkl')
train = pd.read_csv('./raw/train.csv')
test = pd.read_csv('./raw/test.csv')

In [85]:
test_df = pd.merge(test, products, on='partnumber', how='left')
test_df.head()

,session_id,date,timestamp_local,user_id,country,partnumber,device_type,pagetype,discount,embedding,color_id,cod_section,family
0,746,2024-06-15,2024-06-15 18:36:47.390,NaN,57,1254,1,24.0,0,"[-0.10923082, 1.6806983, -0.18787065, -0.09141...",404,1.0,30
1,746,2024-06-15,2024-06-15 18:37:04.052,NaN,57,32544,1,24.0,0,"[-0.15846375, 2.1865053, -0.14408879, 0.356508...",219,1.0,146
2,746,2024-06-15,2024-06-15 18:37:48.159,NaN,57,12639,1,24.0,0,"[-0.1715896, -0.044618968, -0.12034343, 0.2949...",83,1.0,53
3,746,2024-06-15,2024-06-15 18:38:19.899,NaN,57,18048,1,24.0,0,"[0.08176647, -0.02640042, 0.22054362, -0.16622...",53,1.0,29
4,746,2024-06-15,2024-06-15 18:38:46.492,NaN,57,13295,1,24.0,0,"[-0.16236497, -0.06050226, -0.1673642, -0.2320...",181,1.0,51


In [86]:
test_df.isnull().sum()

session_id             0
date                   0
timestamp_local        0
user_id            23509
country                0
partnumber             0
device_type            0
pagetype               0
discount               0
embedding             80
color_id               0
cod_section           15
family                 0
dtype: int64

In [87]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29275 entries, 0 to 29274
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   session_id       29275 non-null  int64  
 1   date             29275 non-null  object 
 2   timestamp_local  29275 non-null  object 
 3   user_id          5766 non-null   float64
 4   country          29275 non-null  int64  
 5   partnumber       29275 non-null  int64  
 6   device_type      29275 non-null  int64  
 7   pagetype         29275 non-null  float64
 8   discount         29275 non-null  object 
 9   embedding        29195 non-null  object 
 10  color_id         29275 non-null  int32  
 11  cod_section      29260 non-null  float64
 12  family           29275 non-null  int32  
dtypes: float64(3), int32(2), int64(4), object(4)
memory usage: 2.7+ MB


In [88]:
test_df['user_id'] = test_df['user_id'].fillna('invitado')
test_df['user_id'] = test_df['user_id'].apply(lambda x: 'registrado' if isinstance(x, float) else x)
test_df['user_id'].head(5)

0    invitado
1    invitado
2    invitado
3    invitado
4    invitado
Name: user_id, dtype: object

In [89]:
test_df['pagetype'] = test_df['pagetype'].fillna(24)
test_df['pagetype'] = test_df['pagetype'].apply(lambda x: x if x in [24, 8, 6] else 24)

In [90]:
test_df['cod_section'] = test_df['cod_section'].fillna(1)

In [91]:
test_df.drop(columns=['embedding'], inplace =True)

In [92]:
#no quedan valores nulos en el dataframe
test_df.isnull().sum()

session_id         0
date               0
timestamp_local    0
user_id            0
country            0
partnumber         0
device_type        0
pagetype           0
discount           0
color_id           0
cod_section        0
family             0
dtype: int64

In [51]:
#ver si hay desbalanceo en los datos: 
train_df['add_to_cart'].value_counts()
#a priori puede parecer que hay desbalanceo en los datos pero si nos fijamos en las sesiones que acaban en compra y las que no, el dataset esta bastante balanceado 

#sesiones que acabaron en compra: 
sessions_with_1 = train_df[train_df['add_to_cart'] == 1]['session_id'].unique()

#guardamos las sesiones que acabaron en compra en un df:
df_keep = train_df[train_df['session_id'].isin(sessions_with_1)] 

#añadir sesiones que acabaron sin compra (en este caso añadiriamos 2millones y pico)
df_downsample = train_df[~train_df['session_id'].isin(sessions_with_1)]  
n_samples = train_df[train_df['add_to_cart'] == 1].shape[0]
df_downsampled = df_downsample.sample(n=n_samples, random_state=42)

final = pd.concat([df_keep, df_downsampled])
final
#trabajamos con este dataset para reducir un poco de coste en el entrenamiento con menos datos

,session_id,date,timestamp_local,add_to_cart,user_id,country,partnumber,device_type,pagetype,discount,color_id,cod_section,family
7,1222,2024-06-13,2024-06-13 06:22:03.307,0,invitado,57,10763,1,24.0,0,64,2.0,4
8,1222,2024-06-13,2024-06-13 06:17:33.149,0,invitado,57,14731,1,24.0,0,72,2.0,195
9,1222,2024-06-13,2024-06-13 06:17:57.411,0,invitado,57,5249,1,24.0,0,83,1.0,196
10,1222,2024-06-13,2024-06-13 06:17:32.601,0,invitado,57,33017,1,24.0,0,239,2.0,23
11,1222,2024-06-13,2024-06-13 06:19:09.272,1,invitado,57,5249,1,24.0,0,83,1.0,196
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35457300,734218,2024-06-10,2024-06-10 22:39:45.096,0,invitado,34,3122,1,24.0,0,342,1.0,30
9636086,1561478,2024-06-14,2024-06-14 17:17:26.688,0,invitado,57,18319,1,24.0,0,119,1.0,146
40312053,2134619,2024-06-09,2024-06-09 20:15:23.055,0,invitado,29,19089,1,24.0,0,284,1.0,53
19111294,3058995,2024-06-05,2024-06-05 19:14:17.435,0,invitado,34,10673,1,24.0,0,135,1.0,29


In [93]:
#train_df = final.copy()
test_df['hour'] = pd.to_datetime(test_df['timestamp_local']).dt.hour
test_df['weekday'] = pd.to_datetime(test_df['timestamp_local']).dt.weekday
test_df.head()

,session_id,date,timestamp_local,user_id,country,partnumber,device_type,pagetype,discount,color_id,cod_section,family,hour,weekday
0,746,2024-06-15,2024-06-15 18:36:47.390,invitado,57,1254,1,24.0,0,404,1.0,30,18,5
1,746,2024-06-15,2024-06-15 18:37:04.052,invitado,57,32544,1,24.0,0,219,1.0,146,18,5
2,746,2024-06-15,2024-06-15 18:37:48.159,invitado,57,12639,1,24.0,0,83,1.0,53,18,5
3,746,2024-06-15,2024-06-15 18:38:19.899,invitado,57,18048,1,24.0,0,53,1.0,29,18,5
4,746,2024-06-15,2024-06-15 18:38:46.492,invitado,57,13295,1,24.0,0,181,1.0,51,18,5


In [94]:
test_df = test_df.drop(columns=['timestamp_local','date'])

In [95]:
#hour, day, weekday serán franjas de tiempos para poder aplicar one hot optimo
#funcion para la hora:
def convertir_dia(hour):
    if 0 <= hour < 9:
        return 'noche'
    elif 9 <= hour < 14:
        return 'mañana'
    else:
        return 'tarde'


#funcion para los fines de semana 
def convertir_semana(weekday):
    if 1 <= weekday <= 5:
        return 'weekday'
    else:
        return 'weekend'

test_df['hour'] = test_df['hour'].apply(lambda x: convertir_dia(x))
test_df['weekday'] = test_df['weekday'].apply(lambda x: convertir_semana(x))

In [96]:
#categorización de variables: 
categorical =[var for var in test_df.columns if test_df[var].dtype=='O']
numerical = [var for var in test_df.columns if test_df[var].dtype != 'O' and (var != 'partnumber')]
print(f'categorical:',categorical)
print(f'numerical:',numerical)

new_categorical = ['country','device_type','pagetype','cod_section']
test_df[new_categorical] = test_df[new_categorical].apply(lambda x: x.astype('object'))
test_df.info()

categorical = categorical + new_categorical
print(categorical)

categorical: ['user_id', 'discount', 'hour', 'weekday']
numerical: ['session_id', 'country', 'device_type', 'pagetype', 'color_id', 'cod_section', 'family']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29275 entries, 0 to 29274
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   session_id   29275 non-null  int64 
 1   user_id      29275 non-null  object
 2   country      29275 non-null  object
 3   partnumber   29275 non-null  int64 
 4   device_type  29275 non-null  object
 5   pagetype     29275 non-null  object
 6   discount     29275 non-null  object
 7   color_id     29275 non-null  int32 
 8   cod_section  29275 non-null  object
 9   family       29275 non-null  int32 
 10  hour         29275 non-null  object
 11  weekday      29275 non-null  object
dtypes: int32(2), int64(2), object(8)
memory usage: 2.5+ MB
['user_id', 'discount', 'hour', 'weekday', 'country', 'device_type', 'pagetype', 'cod_section']


In [97]:
from sklearn.preprocessing import OneHotEncoder, RobustScaler
encoder = OneHotEncoder()
encoder_cat = encoder.fit_transform(test_df[categorical])
encoded_df = pd.DataFrame(encoder_cat.toarray(),columns=encoder.get_feature_names_out().tolist())
encoded_df

,user_id_invitado,user_id_registrado,discount_0,discount_1,hour_mañana,hour_noche,hour_tarde,weekday_weekday,weekday_weekend,country_25,...,device_type_1,device_type_2,device_type_3,pagetype_6.0,pagetype_8.0,pagetype_24.0,cod_section_1.0,cod_section_2.0,cod_section_3.0,cod_section_4.0
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29270,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
29271,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
29272,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
29273,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [98]:
test_df['index'] = test_df['session_id'].astype(str) + '_' + test_df['partnumber'].astype(str)
test_df.head()

,session_id,user_id,country,partnumber,device_type,pagetype,discount,color_id,cod_section,family,hour,weekday,index
0,746,invitado,57,1254,1,24.0,0,404,1.0,30,tarde,weekday,746_1254
1,746,invitado,57,32544,1,24.0,0,219,1.0,146,tarde,weekday,746_32544
2,746,invitado,57,12639,1,24.0,0,83,1.0,53,tarde,weekday,746_12639
3,746,invitado,57,18048,1,24.0,0,53,1.0,29,tarde,weekday,746_18048
4,746,invitado,57,13295,1,24.0,0,181,1.0,51,tarde,weekday,746_13295


In [99]:
numerical = [var for var in test_df.columns if test_df[var].dtype != 'O' and var != 'session_id' and var != 'add_to_cart']
print(numerical)
#normalizamos estas columnas y mergeamos el df
scaler = RobustScaler()
encoder_num = scaler.fit_transform(test_df[numerical])
encoded_num = pd.DataFrame(encoder_num, columns=numerical)
encoded_num.shape
print(encoded_num.head(10))


['partnumber', 'color_id', 'family']
   partnumber  color_id    family
0   -0.962640  1.084677 -0.421569
1    0.505031  0.338710  0.715686
2   -0.428622 -0.209677 -0.196078
3   -0.174910 -0.330645 -0.431373
4   -0.397852  0.185484 -0.215686
5   -0.581627  0.314516 -0.215686
6   -0.577406 -0.245968  0.715686
7    0.265485 -0.463710  0.254902
8   -0.097751  0.641129 -0.196078
9    0.988625 -0.532258  0.813725


In [100]:
encoded_num

,partnumber,color_id,family
0,-0.962640,1.084677,-0.421569
1,0.505031,0.338710,0.715686
2,-0.428622,-0.209677,-0.196078
3,-0.174910,-0.330645,-0.431373
4,-0.397852,0.185484,-0.215686
...,...,...,...
29270,0.391004,0.705645,1.068627
29271,0.335139,1.068548,-0.196078
29272,-0.389784,0.383065,1.117647
29273,-0.377354,0.883065,0.813725


In [101]:
categorical

['user_id',
 'discount',
 'hour',
 'weekday',
 'country',
 'device_type',
 'pagetype',
 'cod_section']

In [117]:
print(categorical)
#print(new_categorical)
print(numerical)
new = categorical + numerical
print(new)
test_df.drop(columns=['family'], inplace=True, axis = 1) 

['user_id', 'discount', 'hour', 'weekday', 'country', 'device_type', 'pagetype', 'cod_section']
['partnumber', 'color_id', 'family']
['user_id', 'discount', 'hour', 'weekday', 'country', 'device_type', 'pagetype', 'cod_section', 'partnumber', 'color_id', 'family']


In [118]:
test_df

,index
0,746_1254
1,746_32544
2,746_12639
3,746_18048
4,746_13295
...,...
29270,5167195_30113
29271,5167272_28922
29272,5168338_13467
29273,5168492_13732


In [119]:
test_df.reset_index(inplace=True, drop = True)

In [120]:
test_df

,index
0,746_1254
1,746_32544
2,746_12639
3,746_18048
4,746_13295
...,...
29270,5167195_30113
29271,5167272_28922
29272,5168338_13467
29273,5168492_13732


In [121]:
final_test = pd.concat([test_df,encoded_num,encoded_df], axis = 1)
final_test.set_index('index', inplace= True)
final_test

,partnumber,color_id,family,user_id_invitado,user_id_registrado,discount_0,discount_1,hour_mañana,hour_noche,hour_tarde,...,device_type_1,device_type_2,device_type_3,pagetype_6.0,pagetype_8.0,pagetype_24.0,cod_section_1.0,cod_section_2.0,cod_section_3.0,cod_section_4.0
index,,,,,,,,,,,,,,,,,,,,,
746_1254,-0.962640,1.084677,-0.421569,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
746_32544,0.505031,0.338710,0.715686,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
746_12639,-0.428622,-0.209677,-0.196078,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
746_18048,-0.174910,-0.330645,-0.431373,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
746_13295,-0.397852,0.185484,-0.215686,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167195_30113,0.391004,0.705645,1.068627,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
5167272_28922,0.335139,1.068548,-0.196078,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
5168338_13467,-0.389784,0.383065,1.117647,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [122]:
final_test.to_parquet('checkpoint4_test.parquet', engine='pyarrow', compression='snappy')